# ENIGH Database

@roman

26 June, 2024

Code to create an ETL of the Enigh database, which is a database of the National Institute of Statistics and Geography (INEGI) of Mexico. The database contains information on the income and expenses of Mexican households.

In [1]:
import pandas as pd
import geo
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm import tqdm

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
# Settings
# show 100 columns in pandas
pd.set_option('display.max_columns', 500)

---
# Concentrado Hogar

The data diccionary is found [here](https://www.inegi.org.mx/rnm/index.php/catalog/685/data-dictionary/F28?file_name=concentradohogar)

## S1: Extract & Transform

In [3]:
# function to extract, transform and load each enigh file
def weighted_mean(df, cols, weight_col):
    return (df[cols].mul(df[weight_col], axis=0).sum() / df[weight_col].sum())


def weighted_mode(df, columns_to_mode, weighting_column):
    # Initialize a DataFrame to store the mode values
    mode_values = pd.DataFrame(columns=['Column', 'Weighted_Mode'])

    # Calculate weighted mode for each column in columns_to_mode
    for col in columns_to_mode:
        # Group by the column values and calculate weighted counts
        weighted_counts = df.groupby(col)[weighting_column].sum()

        # Find the index of maximum weighted count
        mode_index = weighted_counts.idxmax()

        # Get the mode value
        mode_value = df.loc[df[col] == mode_index, col].iloc[0]

        # Append mode value to mode_values DataFrame
        mode_values.loc[len(mode_values)] = [col, mode_value]

    # return a Series
    return mode_values.set_index('Column')['Weighted_Mode']


def get_statistics(df, cols_mode, cols_mean, weight_col):
    # s1: get weighted mode
    mode_values = weighted_mode(df, cols_mode, weight_col)

    # s2: get weighted mean
    mean_values = weighted_mean(df, cols_mean, weight_col)

    # s3: get total viviendas
    total_hogares = df[weight_col].sum()

    # return all the values in a pd series
    melt_values = pd.concat([mode_values, mean_values])
    melt_values['total_hogares'] = total_hogares

    return melt_values


def get_enigh_concentrado_hogar(file, cols_mode, cols_mean):
    # s1:read file
    df = pd.read_csv(file, na_values=[' '])
    
    # s2: subset and wrangle
    df = (
        df
        .query("foliohog == 1")
        .reset_index(drop=True)
        .assign(
            ubica_geo=lambda x: x["ubica_geo"].astype(str).str.zfill(5)
        )
    )

    # s3: get weighted mean
    table = (
        df
        .groupby("ubica_geo", as_index=False)
        .apply(
            lambda x: get_statistics(x, cols_mode, cols_mean, "factor"),
            include_groups=False
        )
    )
    
    return table


In [4]:
# get params
years_enigh = [2018, 2020, 2022]

cols_mode = [
    'est_dis', 'clase_hog', 'sexo_jefe',
]
cols_mean = [
    'tam_loc', 'est_socio',  'edad_jefe',
    'educa_jefe', 'tot_integ', 'hombres', 'mujeres',
    'mayores', 'menores', 'p12_64', 'p65mas', 'ocupados',
    'percep_ing', 'perc_ocupa', 'ing_cor', 'ingtrab',
    'trabajo', 'sueldos', 'horas_extr', 'comisiones',
    'aguinaldo', 'indemtrab', 'otra_rem', 'remu_espec',
    'negocio', 'noagrop', 'industria', 'comercio', 'servicios',
    'agrope', 'agricolas', 'pecuarios', 'reproducc', 'pesca',
    'otros_trab', 'rentas', 'utilidad', 'arrenda', 'transfer',
    'jubilacion', 'becas', 'donativos', 'remesas', 'bene_gob',
    'transf_hog', 'trans_inst', 'estim_alqu', 'otros_ing', 'gasto_mon',
    'alimentos', 'ali_dentro', 'cereales', 'carnes', 'pescado',
    'leche', 'huevo', 'aceites', 'tuberculo', 'verduras', 'frutas',
    'azucar', 'cafe', 'especias', 'otros_alim', 'bebidas', 'ali_fuera',
    'tabaco', 'vesti_calz', 'vestido', 'calzado', 'vivienda',
    'alquiler', 'pred_cons', 'agua', 'energia', 'limpieza', 'cuidados',
    'utensilios', 'enseres', 'salud', 'atenc_ambu', 'hospital',
    'medicinas', 'transporte', 'publico', 'foraneo', 'adqui_vehi',
    'mantenim', 'refaccion', 'combus', 'comunica', 'educa_espa',
    'educacion', 'esparci', 'paq_turist', 'personales', 'cuida_pers',
    'acces_pers', 'otros_gas', 'transf_gas', 'percep_tot', 'retiro_inv',
    'prestamos', 'otras_perc', 'ero_nm_viv', 'ero_nm_hog', 'erogac_tot',
    'cuota_viv', 'mater_serv', 'material', 'servicio', 'deposito',
    'prest_terc', 'pago_tarje', 'deudas', 'balance', 'otras_erog',
]

FILE_TEMPLATE = 'enigh{enigh_year}_ns_concentradohogar_csv.zip'
ENIGH_ROOT = "../../data/catalogues/adamuz_data/enigh_{enigh_year}/"

In [5]:
# get all tables
list_tables = []
for year in tqdm(years_enigh):
    # get file
    dir_file = ENIGH_ROOT.format(enigh_year=year)
    file = dir_file + FILE_TEMPLATE.format(enigh_year=year)

    # unzip
    os.system(f"unzip -o {file} -d {dir_file}")

    # get table
    table = get_enigh_concentrado_hogar(
        dir_file + 'concentradohogar.csv',
        cols_mode,
        cols_mean
        )

    # add year
    table["year"] = year

    # append
    list_tables.append(table)


  0%|          | 0/3 [00:00<?, ?it/s]

Archive:  ../../data/catalogues/adamuz_data/enigh_2018/enigh2018_ns_concentradohogar_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2018/concentradohogar.csv  
  inflating: ../../data/catalogues/adamuz_data/enigh_2018/nota_bases_datos_enigh2018_ns.txt  


 33%|███▎      | 1/3 [00:09<00:18,  9.14s/it]

Archive:  ../../data/catalogues/adamuz_data/enigh_2020/enigh2020_ns_concentradohogar_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2020/concentradohogar.csv  


 67%|██████▋   | 2/3 [00:18<00:09,  9.24s/it]

Archive:  ../../data/catalogues/adamuz_data/enigh_2022/enigh2022_ns_concentradohogar_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2022/concentradohogar.csv  


100%|██████████| 3/3 [00:27<00:00,  9.12s/it]


In [6]:
# concat
df_concentradohogar_all = (
    pd.concat(list_tables)
    .sort_values(["ubica_geo", "year"], ignore_index=True)
    )
df_concentradohogar_all.shape

(3218, 123)

## S2: EDA

In [7]:
# see sample
df_concentradohogar_all.head()

,ubica_geo,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares,year
0,01001,2.0,2.0,1.0,1.313242,2.830593,50.089514,6.479373,3.721097,1.780415,1.940683,2.999458,0.721640,2.691494,0.307964,1.803984,2.391824,1.774901,63956.301534,43070.626604,37697.351960,31202.010734,495.394570,1317.133005,2206.442086,252.169473,852.605979,1371.596112,3683.768570,3046.691033,415.123858,1268.234312,1363.332863,637.077537,0.783675,633.157600,3.136262,0.0,1689.506073,4540.471227,3606.090472,934.380755,9947.056421,5419.655226,159.891602,989.171367,295.724280,299.683788,1599.486996,1183.443163,6351.823899,46.323383,39119.615453,11954.944213,8940.815992,1409.190847,1791.346954,141.805814,1001.070483,256.830314,64.707445,116.593703,888.009195,411.459735,58.341930,74.699891,70.812195,1791.369493,864.577994,2928.156080,85.972141,1740.256643,1088.568451,651.688192,3114.482487,1146.301370,175.330648,644.507169,1148.343300,2429.241635,1727.998294,274.058488,427.184852,1639.883555,1227.447356,197.869469,214.566731,8381.883266,1171.234546,335.669626,1267.892256,3749.066387,355.853740,3393.212646,1858.020452,5514.063280,3462.857938,1320.678289,730.527053,2968.421188,2140.535891,117.545545,710.339753,1376.439186,4509.766233,906.226031,512.698217,1570.168785,10.959857,1509.713343,8311.227510,1231.299092,479.004866,209.606931,269.397936,2451.678069,257.427359,2074.223392,787.065174,316.650080,713.879477,234164.0,2018
1,01001,2.0,2.0,1.0,1.191967,2.848673,50.307018,6.518359,3.511532,1.670060,1.841472,2.877479,0.634053,2.578268,0.299212,1.696833,2.295080,1.642091,60912.502414,39994.510208,36177.707945,30166.188042,278.128582,943.278159,2202.367386,323.549054,1224.366678,1039.830043,2677.040577,2616.953301,378.597045,837.298788,1401.057468,60.087276,2.420698,57.607004,0.059573,0.0,1139.761686,3339.710125,2725.802014,613.908110,10562.410583,6146.652668,104.825597,1051.749094,361.698340,898.020269,1472.286234,527.178381,6937.108693,78.762806,37361.636895,12577.730151,10739.532593,1660.287784,1981.896889,180.714402,1105.435781,366.783710,92.778479,130.161860,1079.879732,523.272579,81.819202,84.717259,85.751375,2324.198133,1041.835407,1720.387090,117.810468,1351.179617,878.251851,472.927765,3523.484832,1395.231161,184.045724,789.587590,1154.620356,2287.033608,1552.910254,271.907183,462.216171,1778.678539,1277.235951,174.258678,327.183910,8257.743571,929.733346,119.353027,1820.589293,3162.720259,342.765384,2819.954876,2225.347646,3700.139892,2536.653595,776.041132,387.445165,2749.710175,1969.424468,112.600106,667.685601,1135.936510,4979.077693,1640.427443,570.140620,1558.225213,42.339486,1167.944931,10520.616231,1834.717706,532.231593,265.539319,266.692273,4694.411677,129.864933,1556.153613,757.984491,463.801093,551.451125,267473.0,2020
2,01001,2.0,2.0,1.0,1.260126,2.843422,50.853007,6.667584,3.485117,1.630896,1.854221,2.844748,0.640369,2.536844,0.307904,1.688633,2.319125,1.663681,85273.975500,54618.627742,46768.736235,3881

In [8]:
# shape
df_concentradohogar_all.shape

(3218, 123)

In [9]:
# see empty values
df_concentradohogar_all.isnull().sum()[df_concentradohogar_all.isnull().sum() > 0]

Series([], dtype: int64)

In [10]:
# count # of ubica_geo
df_concentradohogar_all["ubica_geo"].value_counts().value_counts()

count
3    664
1    502
2    362
Name: count, dtype: int64

In [11]:
# describe
df_concentradohogar_all.describe()

,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares,year
count,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000
mean,306.894655,2.024549,1.018334,3.126627,1.738642,51.777733,5.098020,3.626452,1.756526,1.869925,2.874127,0.752324,2.516746,0.357382,1.737205,2.361901,1.659605,42832.739771,27342.263322,21022.137858,18360.965488,142.262048,387.370905,918.352346,100.772285,375.463892,736.950893,5144.973552,3571.022166,965.558829,1307.249235,1298.214102,1573.951386,839.207739,648.441564,35.628280,50.673804,1175.151912,2560.332132,2155.669740,404.662392,8356.073000,2834.563251,69.764822,1111.201224,842.251982,1682.114427,1294.960880,521.216414,4540.223875,33.847443,26978.456638,10722.503389,9178.216533,1815.776905,1985.935418,191.479814,810.754164,399.759513,179.801073,158.835174,1166.499288,364.515518,132.162500,100.298404,85.964536,1004.160293,782.273934,1500.582380,43.704476,1089.014242,663.350024,425.664218,2143.950353,492.101690,105.836001,233.313465,1312.699197,1745.702951,1326.950768,154.707186,264.044997,1090.296698,805.042192,113.984523,171.269983,5031.318885,1135.877534,138.677962,654.763285,1912.605288,190.388716,1722.216572,1189.394816,2289.396380,1641.266439,485.301510,162.828431,2108.788971,1643.044461,70.585569,395.158941,757.484768,3404.853373,887.634839,423.186890,563.459168,41.937477,1488.634998,5091.985670,281.405813,382.341654,246.206399,136.135255,2710.776539,81.868545,453.112205,378.826458,479.237772,324.416685,32945.253263,2020.084525
std,145.922871,0.170082,0.134178,0.942440,0.

## S3: Explicit Nulls

In [12]:
# generate explicit nulls for ubica_geo and year
index_cols = ["ubica_geo", "year"]
new_index = pd.MultiIndex.from_product(
    [df_concentradohogar_all["ubica_geo"].unique(), df_concentradohogar_all["year"].unique()],
    names=index_cols
)

# reindex
df_concentradohogar_all = (
    df_concentradohogar_all
    .set_index(index_cols)
    .reindex(new_index)
    .reset_index()
    )

In [13]:
# count # of ubica_geo
df_concentradohogar_all["ubica_geo"].value_counts().value_counts()

count
3    1528
Name: count, dtype: int64

In [14]:
# see ubica geo 01004
df_concentradohogar_all.query("ubica_geo == '01004'")

,ubica_geo,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
9,01004,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,01004,2020,11.0,2.0,1.0,4.0,2.0,53.095238,4.952381,4.666667,2.095238,2.571429,3.523810,1.142857,2.952381,0.571429,1.666667,3.095238,1.619048,50108.640000,29205.945714,20554.909048,16689.658571,16.304286,0.0,1425.112381,0.0,863.847619,1559.986190,6453.678571,3989.440000,0.0,58.229524,3931.210476,2464.238571,52.406667,2411.831905,0.0,0.0,2197.358095,0.0,0.0,0.0,17151.685238,6183.724286,58.229524,579.967619,3232.916667,2820.648571,2437.044762,1839.153810,3442.391429,308.617619,29510.594286,13492.470952,11801.460952,2150.163333,1509.784286,0.000000,1468.144762,726.722857,230.506667,200.199524,1711.491905,512.132381,245.506667,91.223333,83.874762,1893.665238,978.045238,1678.765238,12.244762,815.899524,440.392381,375.507143,2110.618571,69.124286,229.52381,320.000000,1491.970476,2548.000000,2275.535238,253.831429,18.633333,405.970000,376.810000,10.993333,18.166667,5659.382381,745.705238,10.946667,1271.738571,2076.111429,71.505238,2004.606190,1554.880476,1202.63,493.482857,476.228095,232.919048,2283.779524,1708.030476,31.210000,544.539048,991.843333,2058.328571,638.197619,698.756667,15.139048,0.0,706.235238,6272.042857,0.0,1411.489524,1276.396667,135.092857,4227.480000,451.396667,0.0,0.0,0.0,181.676667,1155.0
11,01004,2022,11.0,2.0,1.0,4.0,2.0,55.555556,4.166667,4.500000,2.111111,2.388889,3.333333,1.166667,2.833333,0.500000,1.777778,2.666667,1.722222,42181.936667,32637.283333,26569.804444,23682.403333,288.609444,0.0,1494.448333,0.0,179.971111,924.372222,4776.420556,1205.441667,0.0,41.331111,1164.110556,3570.978889,972.763889,2598.215000,0.0,0.0,1291.058333,0.0,0.0,0.0,6420.463333,0.000000,132.596667,1272.037222,379.229444,2604.846111,1673.525000,358.228889,3124.190000,0.000000,26342.328889,10408.444444,9682.019444,2974.238333,949.277778,117.856667,541.420556,427.138333,94.998889,210.711667,1291.761667,419.638889,47.142222,7.142778,14.285000,1331.424444,1254.982222,610.711667,115.713333,847.595556,425.861111,421.734444,2041.126667,166.666667,29.37500,423.333333,1421.751667,1961.920556,1568.127222,173.911667,219.881667,1284.121667,1086.679444,0.000000,197.442222,4915.933889,1071.417778,70.765000,0.000000,2943.806667,208.862778,2734.943889,829.944444,1176.49,657.202778,329.070000,190.217222,3333.295000,2838.871111,2.825556,491.598333,373.401111,637.091111,0.000000,0.000000,0.000000,0.0,637.091111,1127.683333,0.0,2.732222,2.732222,0.00

In [15]:
# fill na with strategy ffill and then bfill by ubica_geo
df_concentradohogar_all = (
    df_concentradohogar_all
    .sort_values(["ubica_geo", "year"])
    .groupby("ubica_geo")
    .apply(lambda x: x.ffill().bfill())
    .reset_index(drop=True)
    )

/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_24320/796339476.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.ffill().bfill())


In [16]:
# see ubica geo 01004
df_concentradohogar_all.query("ubica_geo == '01004'")

,ubica_geo,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
9,01004,2018,11.0,2.0,1.0,4.0,2.0,53.095238,4.952381,4.666667,2.095238,2.571429,3.523810,1.142857,2.952381,0.571429,1.666667,3.095238,1.619048,50108.640000,29205.945714,20554.909048,16689.658571,16.304286,0.0,1425.112381,0.0,863.847619,1559.986190,6453.678571,3989.440000,0.0,58.229524,3931.210476,2464.238571,52.406667,2411.831905,0.0,0.0,2197.358095,0.0,0.0,0.0,17151.685238,6183.724286,58.229524,579.967619,3232.916667,2820.648571,2437.044762,1839.153810,3442.391429,308.617619,29510.594286,13492.470952,11801.460952,2150.163333,1509.784286,0.000000,1468.144762,726.722857,230.506667,200.199524,1711.491905,512.132381,245.506667,91.223333,83.874762,1893.665238,978.045238,1678.765238,12.244762,815.899524,440.392381,375.507143,2110.618571,69.124286,229.52381,320.000000,1491.970476,2548.000000,2275.535238,253.831429,18.633333,405.970000,376.810000,10.993333,18.166667,5659.382381,745.705238,10.946667,1271.738571,2076.111429,71.505238,2004.606190,1554.880476,1202.63,493.482857,476.228095,232.919048,2283.779524,1708.030476,31.210000,544.539048,991.843333,2058.328571,638.197619,698.756667,15.139048,0.0,706.235238,6272.042857,0.0,1411.489524,1276.396667,135.092857,4227.480000,451.396667,0.0,0.0,0.0,181.676667,1155.0
10,01004,2020,11.0,2.0,1.0,4.0,2.0,53.095238,4.952381,4.666667,2.095238,2.571429,3.523810,1.142857,2.952381,0.571429,1.666667,3.095238,1.619048,50108.640000,29205.945714,20554.909048,16689.658571,16.304286,0.0,1425.112381,0.0,863.847619,1559.986190,6453.678571,3989.440000,0.0,58.229524,3931.210476,2464.238571,52.406667,2411.831905,0.0,0.0,2197.358095,0.0,0.0,0.0,17151.685238,6183.724286,58.229524,579.967619,3232.916667,2820.648571,2437.044762,1839.153810,3442.391429,308.617619,29510.594286,13492.470952,11801.460952,2150.163333,1509.784286,0.000000,1468.144762,726.722857,230.506667,200.199524,1711.491905,512.132381,245.506667,91.223333,83.874762,1893.665238,978.045238,1678.765238,12.244762,815.899524,440.392381,375.507143,2110.618571,69.124286,229.52381,320.000000,1491.970476,2548.000000,2275.535238,253.831429,18.633333,405.970000,376.810000,10.993333,18.166667,5659.382381,745.705238,10.946667,1271.738571,2076.111429,71.505238,2004.606190,1554.880476,1202.63,493.482857,476.228095,232.919048,2283.779524,1708.030476,31.210000,544.539048,991.843333,2058.328571,638.197619,698.756667,15.139048,0.0,706.235238,6272.042857,0.0,1411.489524,1276.396667,135.092857,4227.480000,451.396667,0.0,0.0,0.0,181.676667,1155.0
11,01004,2022,11.0,2.0,1.0,4.0,2.0,55.555556,4.166667,4.500000,2.111111,2.388889,3.333333,1.166667,2.833333,0.500000,1.777778,2.666667,1.722222,42181.936667,32637.283333,26569.804444,23682.403333,288.609444,0.0,1494.448333,0.0,179.971111,924.372222,4776.420556,1205.441667,0.0,41.331111,1164.110556,3570.978889,972.763889,2598.215000,0.0,0.0,1291.058333,0.0,0.0,0.0,6420.463333,0.000000,132.596667,1272.037222,379.2294

## S4: Save

In [17]:
# see shape
df_concentradohogar_all.shape

(4584, 123)

In [18]:
# save
dir_save = "../../data/interim/"
df_concentradohogar_all.to_csv(dir_save + "concentrado_hogar_enigh.csv")

---
# Vivienda

INEGI's data diccionary is found [here](https://www.inegi.org.mx/rnm/index.php/catalog/685/data-dictionary/F13?file_name=viviendas)

Notes:
- A Hogar can have multiple viviendas

## S1: Extract & Transform

In [19]:
# function to extract, transform and load each enigh file
def weighted_mode(df, columns_to_mode, weighting_column):
    # Initialize a DataFrame to store the mode values
    mode_values = pd.DataFrame(columns=['Column', 'Weighted_Mode'])

    # Calculate weighted mode for each column in columns_to_mode
    for col in columns_to_mode:
        # Group by the column values and calculate weighted counts
        weighted_counts = df.groupby(col)[weighting_column].sum()

        # Find the index of maximum weighted count
        mode_index = weighted_counts.idxmax()

        # Get the mode value
        mode_value = df.loc[df[col] == mode_index, col].iloc[0]

        # Append mode value to mode_values DataFrame
        mode_values.loc[len(mode_values)] = [col, mode_value]

    # return a Series
    return mode_values.set_index('Column')['Weighted_Mode']


def weighted_mean(df, cols, weight_col):
    return (df[cols].mul(df[weight_col], axis=0).sum() / df[weight_col].sum())


def get_statistics(df, cols_mode, cols_mean, weight_col):
    # s1: get weighted mode
    mode_values = weighted_mode(df, cols_mode, weight_col)

    # s2: get weighted mean
    mean_values = weighted_mean(df, cols_mean, weight_col)

    # s3: get total viviendas
    total_viviendas = df[weight_col].sum()

    # return all the values in a pd series
    melt_values = pd.concat([mode_values, mean_values])
    melt_values['total_viviendas'] = total_viviendas

    return melt_values


def get_enigh_vivienda(file, cols_mode, cols_mean):
    # s1:read file
    df = pd.read_csv(file, na_values=[' '])

    # s2: wrangle
    yes_no_columns = [
        'cocina',
        'cocina_dor',
        'excusado',
        'uso_compar',
        'biodigest',
        'combustible',
        'estufa_chi',
        'lavadero',
        'fregadero',
        'regadera',
        'tinaco_azo',
        'cisterna',
        'pileta',
        'calent_sol',
        'calent_gas',
        'medidor_luz',
        'bomba_agua',
        'tanque_gas',
        'aire_acond',
        'calefacc',
        'pago_mesp',
        'viv_usada'
    ]
    # map 1 for 1 and 2 for 0 (yes and no)
    df[yes_no_columns] = df[yes_no_columns].replace({2: 0})
    # ubica_geo
    df["ubica_geo"] = df["ubica_geo"].astype(str).str.zfill(5)

    # s3: get statistics
    table = (
        df
        .groupby("ubica_geo", as_index=False)
        .apply(
            lambda x: get_statistics(x, cols_mode, cols_mean, "factor"),
            include_groups=False
        )
    )

    return table


In [20]:
# get params
years_enigh = [2018, 2020, 2022]
cols_mode = [
    'mat_pared', 'mat_techos', 'mat_pisos', 'disp_agua',
    'combustible', 'eli_basura', 'tenencia', 'tipo_adqui',
    'tipo_adqui', 'tipo_finan', 'escrituras', 'disp_elect',
    'tipo_viv'
]
cols_mean = [
    'regadera', 'pago_mesp', 'tinaco_azo',
    'lavadero', 'procaptar', 'tot_resid', 'antiguedad',
    'bano_comp', 'calent_sol', 'cocina',
    'aire_acond', 'tot_hom', 'cocina_dor', 'renta',
    'fregadero', 'focos_inca', 'sanit_agua',
    'uso_compar', 'medidor_luz', 'est_socio',
    'tot_muj', 'dotac_agua', 'bano_regad', 'estufa_chi',
    'tam_loc', 'tanque_gas', 'focos_ahor',
    'cisterna', 'cuart_dorm', 'drenaje', 'excusado', 'pileta',
    'num_cuarto', 'calent_gas', 'calefacc',
    'bano_excus', 'pago_viv',
    'bomba_agua', 'viv_usada', 'biodigest', 'tot_hog',
    'estim_pago'
]

FILE_TEMPLATE = 'enigh{enigh_year}_ns_viviendas_csv.zip'
ENIGH_ROOT = "../../data/catalogues/adamuz_data/enigh_{enigh_year}/"

In [21]:
# get all tables
list_tables = []
for year in tqdm(years_enigh):
    # get file
    dir_file = ENIGH_ROOT.format(enigh_year=year)
    file = dir_file + FILE_TEMPLATE.format(enigh_year=year)

    # unzip
    os.system(f"unzip -o {file} -d {dir_file}")

    # get table
    table = get_enigh_vivienda(dir_file + 'viviendas.csv', cols_mode, cols_mean)

    # add year
    table["year"] = year

    # append
    list_tables.append(table)

  0%|          | 0/3 [00:00<?, ?it/s]

Archive:  ../../data/catalogues/adamuz_data/enigh_2018/enigh2018_ns_viviendas_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2018/viviendas.csv  
  inflating: ../../data/catalogues/adamuz_data/enigh_2018/nota_bases_datos_enigh2018_ns.txt  


/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_24320/892717475.py:47: DtypeWarning: Columns (4,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, na_values=[' '])
 33%|███▎      | 1/3 [00:39<01:18, 39.23s/it]

Archive:  ../../data/catalogues/adamuz_data/enigh_2020/enigh2020_ns_viviendas_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2020/viviendas.csv  


/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_24320/892717475.py:47: DtypeWarning: Columns (1,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, na_values=[' '])
 67%|██████▋   | 2/3 [01:08<00:33, 33.21s/it]

Archive:  ../../data/catalogues/adamuz_data/enigh_2022/enigh2022_ns_viviendas_csv.zip
  inflating: ../../data/catalogues/adamuz_data/enigh_2022/viviendas.csv  


/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_24320/892717475.py:47: DtypeWarning: Columns (4,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file, na_values=[' '])
100%|██████████| 3/3 [01:37<00:00, 32.37s/it]


In [22]:
# concat
df_viviendas_all = (
    pd.concat(list_tables)
    .sort_values(["ubica_geo", "year"], ignore_index=True)
    )
df_viviendas_all

,ubica_geo,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,year
0,01001,8.0,10.0,3.0,1.0,3.0,3.0,4.0,1.0,1.0,1,1.0,1,1,0.945602,0.362238,0.782853,0.895975,0.0,3.781875,19.145343,1.265395,0.263397,0.979702,0.015126,1.806226,0.023304,385.384175,0.882548,1.506320,1.069981,0.029129,0.972374,2.830593,1.975650,1.091257,0.010228,0.001665,1.313242,0.146555,7.696495,0.493936,2.460400,1.044960,0.996631,0.133018,4.529099,0.515284,0.010715,0.266219,442.032669,0.416913,0.166781,0.014665,1.019717,2165.050435,234164,2018
1,01001,8.0,10.0,3,1.0,3.0,3.0,4.0,1.0,1.0,1,1.0,1,1,0.962908,0.385572,0.805143,0.903710,0.0,3.555936,18.578944,1.301716,0.287558,0.988414,0.017228,1.693341,0.020858,484.117574,0.898580,1.067147,1.066227,0.022514,0.982148,2.848673,1.862595,1.177386,0.007960,0.003885,1.191967,0.152516,7.850508,0.475506,2.355871,1.009949,0.996949,0.098937,4.408034,0.492554,0.012072,0.198125,670.729401,0.391329,0.161022,0.010222,1.016633,2361.336658,267473,2020
2,01001,8.0,10.0,3,1.0,3,3.0,4.0,1.0,1.0,1,1.0,1,1,0.970420,0.386612,0.784273,0.905320,0.0,3.514820,20.415628,1.364741,0.350969,0.986751,0.027798,1.645770,0.018158,594.349359,0.916743,0.690979,1.054902,0.015830,0.972788,2.843422,1.869050,1.180679,0.005229,0.004247,1.260126,0.167579,8.915832,0.438922,2.393433,1.033536,0.998091,0.120005,4.433587,0.439448,0.017268,0.239865,711.650391,0.403757,0.179857,0.029685,1.010339,3013.785845,267713,2022
3,01002,8.0,10.0,2.0,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1,1,0.858142,0.033360,0.825390,0.900527,0.0,3.604238,19.959440,0.925979,0.559116,0.966640,0.000000,1.983979,0.110221,35.956195,0.604847,2.070980,1.075745,0.064997,0.978503,2.000000,1.620260,1.121476,0.032752,0.000000,3.502535,0.193673,3.612148,0.108497,2.040256,1.031637,0.989252,0.097141,3.856520,0.383695,0.000000,0.225918,0.000000,0.066112,0.044109,0.010748,1.000000,1048.428311,9862,2018
4,01002,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1,1,0.835095,0.026823,0.828621,0.833510,0.0,4.468816,22.767310,0.884514,0.607426,0.990618,0.008060,2.112183,0.018763,20.989693,0.569371,2.511364,1.166226,0.051401,0.951506,2.000000,2.356633,2.066332,0.021934,0.021010,4.000000,0.085359,4.049154,0.215777,2.379889,1.227669,0.979651,0.229783,4.209567,0.219212,0.008060,0.231501,0.000000,0.203356,0.029070,0.000000,1.027484,1159.791226,7568,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3213,32056,8.0,10.0,3.0,1.0,3.0,1.0,4.0,1.0,1.0,5,1.0,1,1,0.988622,0.307692,0.976436,0.669133,0.0,3.634024,20.698178,1.414077,0.417638,0.993963,0.000000,1.681708,0.000000,435.273334,0.899652,1.926805,1.035972,0.006065,0.993434,3.121463,1.952316,2.024788,0.011963,0.000000,1.000000,0.240840,7.018640,0.221616,2.378717,1.000000,1.000000,0.054389,4.806148,0.810238,0.000000,0.247016,363.107525,0.197357,0.084880,0.005648,1.010433,2008.997079,35945,2018
3214,32056,8.0,10.0,3,1.0,3.0,1.0,4.0,2.0,2.0,5,1.0,1,1,0.914939,0.214437,0.927101,0.696612,0.0,3.529814,21.250086,1.260574,0.583608,0.980378,0.014230,1.642572,0.019228,358.733813,0.821385,1.129967,1.119824,0.019942,0.968511,2.869344,1.887242,1.902826,0.018415,0.005072,1.304299,0.267098,7.814270,0.217613,2.318750,1.071003,0.994928,0.135900,4.486508,0.682087,0.014058,0.343468,295.051455,0.215742,0.058841,0.004481,1.021641,2243.818012,40618,2020
3215,32056,8.0,10.0,3.0,1.0,3.0,1.0,4.0,3.0,3.0,5,1.0,1,1,0.

## S2: EDA

In [23]:
# see sample
df_viviendas_all.head()

,ubica_geo,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,year
0,01001,8.0,10.0,3.0,1.0,3.0,3.0,4.0,1.0,1.0,1,1.0,1,1,0.945602,0.362238,0.782853,0.895975,0.0,3.781875,19.145343,1.265395,0.263397,0.979702,0.015126,1.806226,0.023304,385.384175,0.882548,1.506320,1.069981,0.029129,0.972374,2.830593,1.975650,1.091257,0.010228,0.001665,1.313242,0.146555,7.696495,0.493936,2.460400,1.044960,0.996631,0.133018,4.529099,0.515284,0.010715,0.266219,442.032669,0.416913,0.166781,0.014665,1.019717,2165.050435,234164,2018
1,01001,8.0,10.0,3,1.0,3.0,3.0,4.0,1.0,1.0,1,1.0,1,1,0.962908,0.385572,0.805143,0.903710,0.0,3.555936,18.578944,1.301716,0.287558,0.988414,0.017228,1.693341,0.020858,484.117574,0.898580,1.067147,1.066227,0.022514,0.982148,2.848673,1.862595,1.177386,0.007960,0.003885,1.191967,0.152516,7.850508,0.475506,2.355871,1.009949,0.996949,0.098937,4.408034,0.492554,0.012072,0.198125,670.729401,0.391329,0.161022,0.010222,1.016633,2361.336658,267473,2020
2,01001,8.0,10.0,3,1.0,3,3.0,4.0,1.0,1.0,1,1.0,1,1,0.970420,0.386612,0.784273,0.905320,0.0,3.514820,20.415628,1.364741,0.350969,0.986751,0.027798,1.645770,0.018158,594.349359,0.916743,0.690979,1.054902,0.015830,0.972788,2.843422,1.869050,1.180679,0.005229,0.004247,1.260126,0.167579,8.915832,0.438922,2.393433,1.033536,0.998091,0.120005,4.433587,0.439448,0.017268,0.239865,711.650391,0.403757,0.179857,0.029685,1.010339,3013.785845,267713,2022
3,01002,8.0,10.0,2.0,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1,1,0.858142,0.033360,0.825390,0.900527,0.0,3.604238,19.959440,0.925979,0.559116,0.966640,0.000000,1.983979,0.110221,35.956195,0.604847,2.070980,1.075745,0.064997,0.978503,2.000000,1.620260,1.121476,0.032752,0.000000,3.502535,0.193673,3.612148,0.108497,2.040256,1.031637,0.989252,0.097141,3.856520,0.383695,0.000000,0.225918,0.000000,0.066112,0.044109,0.010748,1.000000,1048.428311,9862,2018
4,01002,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,3.0,5,1.0,1,1,0.835095,0.026823,0.828621,0.833510,0.0,4.468816,22.767310,0.884514,0.607426,0.990618,0.008060,2.112183,0.018763,20.989693,0.569371,2.511364,1.166226,0.051401,0.951506,2.000000,2.356633,2.066332,0.021934,0.021010,4.000000,0.085359,4.049154,0.215777,2.379889,1.227669,0.979651,0.229783,4.209567,0.219212,0.008060,0.231501,0.000000,0.203356,0.029070,0.000000,1.027484,1159.791226,7568,2020


In [24]:
# shape
df_viviendas_all.shape

(3218, 58)

In [25]:
# see empty values
df_viviendas_all.isnull().sum()[df_viviendas_all.isnull().sum() > 0]

Series([], dtype: int64)

In [26]:
# count # of ubica_geo
df_viviendas_all["ubica_geo"].value_counts().value_counts()

count
3    664
1    502
2    362
Name: count, dtype: int64

In [27]:
# describe
df_viviendas_all.describe()

,mat_pared,mat_techos,disp_agua,eli_basura,tenencia,tipo_adqui,tipo_adqui,escrituras,disp_elect,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,year
count,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000,3218.000000
mean,7.872281,8.279366,1.603170,1.633002,3.943443,2.169049,2.169049,1.353014,1.006837,0.632663,0.116859,0.539838,0.833896,0.072518,3.680122,19.966795,0.717635,0.131332,0.907696,0.107184,1.782540,0.060955,168.136975,0.468555,1.124025,1.461249,0.073366,0.919543,1.738642,1.897582,1.527458,0.043776,0.085863,3.126627,0.078632,5.008371,0.152714,2.017363,1.704186,0.967612,0.446672,3.619212,0.270530,0.020671,0.479583,105.376335,0.223503,0.057935,0.023900,1.016293,1546.849737,32945.253263,2020.084525
std,0.428437,2.956521,1.409176,1.206857,0.373886,0.652590,0.652590,0.760988,0.153550,0.290137,0.137805,0.281555,0.204577,0.250589,0.578878,5.926552,0.388612,0.191552,0.110453,0.204979,0.319315,0.058519,321.772576,0.283027,0.738890,0.411315,0.071763,0.112973,0.599341,0.332447,0.764325,0.066133,0.157747,0.942440,0.103955,1.836917,0.174639,0.275016,0.771586,0.066174,0.261569,0.583695,0.249719,0.065202,0.275174,243.101552,0.207830,0.067028,0.047719,0.030076,899.843595,60317.776856,1.624437
min,3.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.823529,3.733333,0.000000,0.000000,0.187500,0.000000,0.823529,0.000000,0.000000,0.000000,0.000000,0.285714,0.000000,0.000000,1.000000,0.500000,0.000000,0.000000,0.000000,1.000000,0.000000,0.032258,0.000000,1.055556,1.000000,0.142857,0.000000,1.684211,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,346.000000,596.000000,2018.000000
25%,8.000000,8.000000,1.000000,1.000000,4.000000,2.000000,2.000000,1.000000,1.000000,0.421113,0.000000,0.300000,0.773755,0.000000,3.315789,15.987030,0.425251,0.000000,0.875000,0.000000,1.578024,0.020557,0.000000,0.227002,0.565231,1.150000,0.023877,0.898095,1.000000,1.686838,1.000501,0.000000,0.000000,2.516317,0.000000,3.782784,0.029041,1.833333,1.086957,0.956431,0.225509,3.244407,0.045455,0.000000,0.252286,0.000000,0.052632,0.000000,0.000000,1.000000,1039.152261,7035.750000,2018.000000
50%,8.000000,10.000000,1.000000,1.000000,4.000000,2.000000,2.000000,1.000000,1.000000,0.697116,0.063959,0.580633,0.918696,0.000000,3.619048,19.370338,0.765071,0.029813,0.947368,0.000000,1.750978,0.048585,60.000000,0.462269,0.992645,1.352941,0.052632,0.954164,2.000000,1.869565,1.262893,0.021841,0.023276,3.298769,0.045455,4.947368,0.092385,2.017716,1.423005,0.997983,0.437706,3.631579,0.210526,0.000000,0.445595,0.000000,0.166667,0.043478,0.000000,1.000000,1340.976532,13650.000000,2020.000000
75%,8.000000,10.000000,2.000000,1.000000,4.000000,3.000000,3.000000,1.000000,1.000000,0.878542,0.190476,0.786549,0.970587,0.000000,3.991653,23.232315,1.018641,0.203478,0.984959,0.095625,1.954545,0.086957,221.486058,0.700000,1.565217,1.666667,0.100950,0.992186,2.000000,2.076226,1.904762,0.055767,0.094195,4.000000,0.111713,6.126788,0.217391,2.195970,2.110828,1.0000

## S3: Explicit Nulls

In [28]:
# generate explicit nulls for ubica_geo and year
index_cols = ["ubica_geo", "year"]
new_index = pd.MultiIndex.from_product(
    [df_viviendas_all["ubica_geo"].unique(), df_viviendas_all["year"].unique()],
    names=index_cols
)

# reindex
df_viviendas_all = (
    df_viviendas_all
    .set_index(index_cols)
    .reindex(new_index)
    .reset_index()
    )

In [29]:
# count # of ubica_geo
df_viviendas_all["ubica_geo"].value_counts().value_counts()

count
3    1528
Name: count, dtype: int64

In [45]:
# remove duplicated columns
df_viviendas_all = df_viviendas_all.loc[:, ~df_viviendas_all.columns.duplicated()]


In [46]:
# see ubica geo 01004
df_viviendas_all.query("ubica_geo == '01004'")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
9,01004,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,01004,2020,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,5,1.0,1.0,1,0.904762,0.047619,0.809524,0.952381,0.0,4.666667,28.666667,1.000000,0.333333,0.904762,0.0,2.095238,0.000000,23.809524,0.666667,1.571429,1.190476,0.047619,0.952381,2.0,2.571429,1.142857,0.047619,0.0,4.0,0.047619,5.047619,0.333333,2.666667,1.000000,1.0,0.238095,4.428571,0.285714,0.000000,0.142857,0.0,0.619048,0.0,0.0,1.0,1185.714286,1155.0
11,01004,2022,8.0,10.0,2,1.0,3,3.0,4.0,3.0,5,1.0,1.0,1,0.833333,0.055556,0.666667,0.666667,0.0,4.500000,15.277778,0.833333,0.222222,0.944444,0.0,2.111111,0.055556,55.555556,0.611111,0.555556,1.277778,0.111111,0.888889,2.0,2.388889,1.000000,0.000000,0.0,4.0,0.000000,4.333333,0.111111,2.111111,1.277778,1.0,0.222222,3.611111,0.166667,0.055556,0.333333,0.0,0.111111,0.0,0.0,1.0,1072.222222,1494.0


In [47]:
# fill na with strategy ffill and then bfill by ubica_geo
df_viviendas_all = (
    df_viviendas_all
    .sort_values(["ubica_geo", "year"])
    .groupby("ubica_geo")
    .apply(lambda x: x.ffill().bfill())
    .reset_index(drop=True)
    )

/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_24320/2231233023.py:6: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  .apply(lambda x: x.ffill().bfill())
/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_24320/2231233023.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x.ffill().bfill())


In [48]:
# see ubica geo 01004
df_viviendas_all.query("ubica_geo == '01004'")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
9,01004,2018,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,5,1.0,1.0,1,0.904762,0.047619,0.809524,0.952381,0.0,4.666667,28.666667,1.000000,0.333333,0.904762,0.0,2.095238,0.000000,23.809524,0.666667,1.571429,1.190476,0.047619,0.952381,2.0,2.571429,1.142857,0.047619,0.0,4.0,0.047619,5.047619,0.333333,2.666667,1.000000,1.0,0.238095,4.428571,0.285714,0.000000,0.142857,0.0,0.619048,0.0,0.0,1.0,1185.714286,1155.0
10,01004,2020,8.0,10.0,2,1.0,3.0,3.0,4.0,3.0,5,1.0,1.0,1,0.904762,0.047619,0.809524,0.952381,0.0,4.666667,28.666667,1.000000,0.333333,0.904762,0.0,2.095238,0.000000,23.809524,0.666667,1.571429,1.190476,0.047619,0.952381,2.0,2.571429,1.142857,0.047619,0.0,4.0,0.047619,5.047619,0.333333,2.666667,1.000000,1.0,0.238095,4.428571,0.285714,0.000000,0.142857,0.0,0.619048,0.0,0.0,1.0,1185.714286,1155.0
11,01004,2022,8.0,10.0,2,1.0,3,3.0,4.0,3.0,5,1.0,1.0,1,0.833333,0.055556,0.666667,0.666667,0.0,4.500000,15.277778,0.833333,0.222222,0.944444,0.0,2.111111,0.055556,55.555556,0.611111,0.555556,1.277778,0.111111,0.888889,2.0,2.388889,1.000000,0.000000,0.0,4.0,0.000000,4.333333,0.111111,2.111111,1.277778,1.0,0.222222,3.611111,0.166667,0.055556,0.333333,0.0,0.111111,0.0,0.0,1.0,1072.222222,1494.0


## S4: Save

In [49]:
# see shape
df_viviendas_all.shape

(4584, 57)

In [50]:
# save
dir_save = "../../data/interim/"
df_viviendas_all.to_csv(dir_save + "viviendas_enigh.csv")

---
# Join with Properties

## S1: Load & Transform

In [55]:
# read parquet
df_properties = (
    pd.read_parquet("../../data/interim/cleaned_data_s4.parquet")
    .assign(
        state_id=lambda x: x['state_id'].astype(str).str.zfill(2),
        municipality_id=lambda x: x['municipality_id'].astype(str).str.zfill(3)
    )
)   


# get original columns
original_columns = df_properties.columns

# see num of rows
df_properties.shape

(1909959, 51)

In [56]:
# see columns cve ent & cve mun
df_properties[["state_id", "municipality_id"]]

,state_id,municipality_id
0,31,050
1,31,041
2,31,050
3,31,041
4,31,050
...,...,...
1909954,01,001
1909955,01,001
1909956,14,120
1909957,13,013


In [57]:
# create 'ubica_geo' column
df_properties["ubica_geo"] = (
    df_properties["state_id"].astype(str).str.zfill(2)
    + df_properties["municipality_id"].astype(str).str.zfill(3)
    )

# see some values
df_properties["ubica_geo"]

0          31050
1          31041
2          31050
3          31041
4          31050
           ...  
1909954    01001
1909955    01001
1909956    14120
1909957    13013
1909958    15081
Name: ubica_geo, Length: 1909959, dtype: object

In [58]:
# get year of fecha avaluo and floor every 2 years
df_properties["year_enigh"] = (
    df_properties["valuation_date"].dt.year
    // 2 * 2
    )

# describe
df_properties["year_enigh"].describe()

count    1.909959e+06
mean     2.020327e+03
std      1.499379e+00
min      2.018000e+03
25%      2.020000e+03
50%      2.020000e+03
75%      2.022000e+03
max      2.022000e+03
Name: year_enigh, dtype: float64

## S2: Join with Concentrado Hogar

In [59]:
# join with df_concentradohogar_all
df_properties = (
    df_properties
    .merge(
        df_concentradohogar_all,
        left_on=["ubica_geo", "year_enigh"],
        right_on=["ubica_geo", "year"],
        how="left"
        )
    )

# see columns
df_properties.shape

(1909959, 175)

In [60]:
# see size
df_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1909959 entries, 0 to 1909958
Columns: 175 entries, half_bathrooms to total_hogares
dtypes: datetime64[ns](1), float32(3), float64(138), int16(1), int64(1), object(3), string(12), uint16(1), uint32(1), uint8(14)
memory usage: 2.3+ GB


In [61]:
# see if there are columns with "_y"
df_properties.filter(like="_x").columns

Index([], dtype='object')

In [62]:
# see how many nan values 
df_properties.loc[:, df_concentradohogar_all.columns].isnull().sum()

ubica_geo           0
year             5669
est_dis          5669
clase_hog        5669
sexo_jefe        5669
                 ... 
pago_tarje       5669
deudas           5669
balance          5669
otras_erog       5669
total_hogares    5669
Length: 123, dtype: int64

In [63]:
# see how many nan values 
df_properties.loc[:, df_concentradohogar_all.columns].isnull().mean()

ubica_geo        0.000000
year             0.002968
est_dis          0.002968
clase_hog        0.002968
sexo_jefe        0.002968
                   ...   
pago_tarje       0.002968
deudas           0.002968
balance          0.002968
otras_erog       0.002968
total_hogares    0.002968
Length: 123, dtype: float64

## S3: Join with Vivienda

In [64]:
# join with df_viviendas_all
df_properties = (
    df_properties
    .drop(columns=["year", "tam_loc", "est_socio"])
    .merge(
        df_viviendas_all,
        left_on=["ubica_geo", "year_enigh"],
        right_on=["ubica_geo", "year"],
        how="left"
        )
    )

# see columns
df_properties.shape

(1909959, 228)

In [65]:
# see size
df_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1909959 entries, 0 to 1909958
Columns: 228 entries, half_bathrooms to total_viviendas
dtypes: datetime64[ns](1), float32(3), float64(187), int16(1), int64(1), object(7), string(12), uint16(1), uint32(1), uint8(14)
memory usage: 3.0+ GB


In [66]:
# see if there are columns with "_y"
df_properties.filter(like="_x").columns

Index([], dtype='object')

In [67]:
# see how many nan values 
df_properties.loc[:, df_viviendas_all.columns].isnull().sum()

ubica_geo             0
year               5669
mat_pared          5669
mat_techos         5669
mat_pisos          5669
disp_agua          5669
combustible        5669
eli_basura         5669
tenencia           5669
tipo_adqui         5669
tipo_finan         5669
escrituras         5669
disp_elect         5669
tipo_viv           5669
regadera           5669
pago_mesp          5669
tinaco_azo         5669
lavadero           5669
procaptar          5669
tot_resid          5669
antiguedad         5669
bano_comp          5669
calent_sol         5669
cocina             5669
aire_acond         5669
tot_hom            5669
cocina_dor         5669
renta              5669
fregadero          5669
focos_inca         5669
sanit_agua         5669
uso_compar         5669
medidor_luz        5669
est_socio          5669
tot_muj            5669
dotac_agua         5669
bano_regad         5669
estufa_chi         5669
tam_loc            5669
tanque_gas         5669
focos_ahor         5669
cisterna        

In [68]:
# see how many nan values 
df_properties.loc[:, df_viviendas_all.columns].isnull().mean()

ubica_geo          0.000000
year               0.002968
mat_pared          0.002968
mat_techos         0.002968
mat_pisos          0.002968
disp_agua          0.002968
combustible        0.002968
eli_basura         0.002968
tenencia           0.002968
tipo_adqui         0.002968
tipo_finan         0.002968
escrituras         0.002968
disp_elect         0.002968
tipo_viv           0.002968
regadera           0.002968
pago_mesp          0.002968
tinaco_azo         0.002968
lavadero           0.002968
procaptar          0.002968
tot_resid          0.002968
antiguedad         0.002968
bano_comp          0.002968
calent_sol         0.002968
cocina             0.002968
aire_acond         0.002968
tot_hom            0.002968
cocina_dor         0.002968
renta              0.002968
fregadero          0.002968
focos_inca         0.002968
sanit_agua         0.002968
uso_compar         0.002968
medidor_luz        0.002968
est_socio          0.002968
tot_muj            0.002968
dotac_agua         0

## S4: See which ubica_geo have nan values

In [69]:
# see how many nan values in ubica_geo
nan_ubica_geo = (
    df_properties
    .loc[:, df_viviendas_all.columns]
    .groupby("ubica_geo")
    .apply(lambda x: x['tipo_viv'].isnull().sum())
    .to_frame()
    .rename(columns={0: "nan_values"})
    .query("nan_values > 0")
    .reset_index()
)
nan_ubica_geo

/var/folders/42/2lkg1sf91wv7mjxw6klfcqtc0000gn/T/ipykernel_24320/604965362.py:6: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda x: x['tipo_viv'].isnull().sum())


,ubica_geo,nan_values
0,07012,1998
1,14051,920
2,15012,8
3,15022,5
4,15055,111
5,15069,26
6,15072,1
7,15073,1155
8,15075,2
9,15093,1


ideas to input missing values:
1. remove rows with missing values
2. fill with mean/median
3. fill with the most common value in the state
4. knn imputer using centroids of the municipalities

## S5: NaN Imputation

### Concentrado Hogar

In [70]:
# to new_index add nan_ubica_geo ubica_geo's
index_cols = ["ubica_geo", "year"]
new_index = pd.MultiIndex.from_product(
    [df_properties["ubica_geo"].unique(), df_properties["year_enigh"].unique()],
    names=index_cols
)

# reindex
df_concentradohogar_all = (
    df_concentradohogar_all
    .set_index(index_cols)
    .reindex(new_index)
    .reset_index()
    )

In [71]:
# see 31100
df_concentradohogar_all.query("ubica_geo == '31100'")

,ubica_geo,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
984,31100,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
985,31100,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986,31100,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# get cve_ent from ubica_geo
df_concentradohogar_all["cve_ent"] = df_concentradohogar_all["ubica_geo"].str[:2]

In [73]:
# get table of median values by cve_ent
table_concentradohogar_medians = (
    df_concentradohogar_all
    .drop(columns=["ubica_geo"])
    .groupby(["cve_ent", "year"], as_index=False)
    .mean()
)
table_concentradohogar_medians

,cve_ent,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
0,01,2018,7.25,2.0,1.0,2.522576,2.414177,47.687678,5.924445,4.231678,2.103663,2.128015,3.225904,1.005775,2.987313,0.238591,1.933839,2.601711,1.908534,57546.501917,42175.722370,37856.587978,31547.447171,605.131393,576.517348,2326.051831,154.959261,1130.499934,1515.981040,2875.245978,2661.243760,369.481068,1433.138650,858.624042,214.002217,2.293808,195.643490,16.064920,0.000000,1443.888415,3355.685716,2615.570958,740.114757,7169.642607,3199.390802,137.395118,633.645283,666.325667,375.672929,1481.425311,675.787496,4828.325341,17.125883,37814.258156,11934.957508,9227.468909,1724.210104,1681.992910,112.469329,985.448146,300.019253,91.806188,145.132712,957.057835,382.807851,69.522183,45.774184,79.061287,1731.165731,921.001197,2632.970715,74.517884,2029.283931,1185.964255,843.319677,2658.848390,802.044744,136.601105,528.640498,1191.562043,2424.583557,1621.714474,247.770211,555.098872,1026.498332,717.805619,110.611874,198.080839,8745.990408,1064.118884,236.550508,2691.796336,3283.818842,252.486822,3031.332020,1469.705838,4993.581514,3085.307053,1084.595370,823.679092,3053.109005,2271.484178,138.913064,642.711763,947.405509,7252.788357,1998.429067,1019.924174,2820.591134,4.061614,1409.782368,7940.738606,1039.656032,360.747981,208.338887,152.409094,2208.098865,161.992612,1103.215315,853.770753,360.464153,1852.792894,71810.500000
1,01,2020,7.75,2.0,1.0,2.449221,2.364563,48.758832,6.015882,3.879883,1.896065,1.983817,3.080848,0.799035,2.854600,0.226248,1.851366,2.460383,1.796398,56350.007168,40087.355633,35701.544646,29465.393712,500.282880,642.235859,2011.113880,755.954370,955.121441,1371.442504,3086.483947,2470.259256,419.270481,905.429470,1145.559306,616.224691,0.863741,615.346057,0.014893,0.000000,1299.327040,2514.508859,2142.143591,372.365267,8730.815872,3230.844841,89.629829,826.649587,1357.597954,747.266949,1682.485452,796.341261,4957.400292,59.926512,33853.781161,12182.777551,10547.309440,1828.628470,1913.509970,130.174343,990.255509,361.068749,109.922133,158.698904,1225.866169,539.889474,85.681711,61.430062,87.498870,1982.642267,1072.042808,1532.868293,102.599817,1335.359985,829.227718,506.132267,2816.261812,902.198784,174.745960,599.311830,1140.005238,2242.630952,1552.295019,223.740292,466.595641,1517.744163,1067.810596,160.856458,289.077109,7092.826849,662.482944,82.318935,1663.511317,2707.404832,323.181176,2384.223657,1977.108820,2972.029507,1940.673436,694.151772,337.204299,2690.676422,2122.897479,93.349568,474.429376,1003.473920,5504.859199,2294.829563,544.879857,1132.624126,18.439067,1514.086586,9609.479342,1057.457097,769.350187,315.337099,454.013088,4986.049829,134.394666,887.047999,777.237696,481.488880,516.452989,84977.750000
2,01,2022,7.75,2.0,1.0,2.679804,2.337955,48.127423,6.184812,3.791812,1.831449,1.960363,2.913784,0.878028,2.702086,0.211698,1.691886,2.405484,1.679086,68725.334446,46849.303706,40985.57513

In [74]:
# append to df_concentradohogar_all for nan values
mask = (
    df_concentradohogar_all['clase_hog'].isnull()
)
columns_to_impute = df_concentradohogar_all.drop(columns=["ubica_geo", "year", "cve_ent"]).columns.copy()

# append to df_concentradohogar_all for nan values
df_concentradohogar_all.loc[mask, columns_to_impute] = (
    df_concentradohogar_all
    .loc[mask, :]
    .merge(
        table_concentradohogar_medians,
        on=["cve_ent", "year"],
        how="left"
        )
    .filter(like="_y")
    .values
)

# sort by ubica_geo and year
df_concentradohogar_all = (
    df_concentradohogar_all
    .sort_values(["ubica_geo", "year"], ignore_index=True)
    )

# see how many nan values
df_concentradohogar_all.isnull().sum()[df_concentradohogar_all.isnull().sum() > 0]

Series([], dtype: int64)

In [75]:
# see 31100
df_concentradohogar_all.query("ubica_geo == '31100'")

,ubica_geo,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares,cve_ent
1023,31100,2018,523.5,2.0,1.0,2.456703,2.136037,47.884917,5.393414,3.787951,1.928931,1.859019,3.043420,0.744530,2.769623,0.273798,2.135098,2.615438,2.119616,48829.923423,35268.822907,30158.612997,25160.049908,261.022478,1355.279482,1250.521360,95.521160,446.907764,1589.310846,3581.597498,3530.590390,885.469929,657.283153,1987.837307,51.007108,8.034997,3.786773,39.185338,0.0,1528.612412,3080.148049,2635.184312,444.963737,6337.237566,3092.755226,64.606151,969.174025,2.304668,405.076266,1361.925334,441.395896,4122.470070,21.244831,30109.082739,11960.566853,10201.588787,1727.076917,2238.691100,75.307536,552.903825,313.925006,76.572678,99.372963,757.031369,224.305888,62.460342,76.621581,132.800940,2424.238730,1440.279913,1721.087436,37.890630,988.841360,621.111680,367.729681,1814.376960,429.100939,54.296103,90.660146,1240.319772,1626.419767,1239.560925,81.348146,305.510696,881.197583,599.048439,173.890520,108.258624,6685.024772,2292.568324,104.673306,886.218412,1880.483958,163.920720,1716.563238,1521.080772,3374.304039,2388.266291,839.749936,146.287811,1805.657499,1510.164102,92.393330,203.100067,972.693908,3438.846770,945.507827,504.581552,643.013262,23.859404,1321.884725,7952.846139,956.530281,197.312960,130.386248,66.926712,3730.836279,378.061268,924.414087,623.302979,187.290210,955.098076,80021.50,31
1024,31100,2020,537.5,2.0,1.0,2.238077,2.122419,46.601857,5.963540,3.622582,1.704812,1.917770,2.841130,0.781453,2.586005,0.255125,1.875707,2.444548,1.814711,42069.208478,28460.115229,23616.203553,19266.157268,296.372213,1052.102110,1088.724343,214.578797,520.460090,1177.808734,3195.503220,3126.251192,503.937563,920.649766,1701.663863,69.252028,9.188451,29.590379,30.473198,0.0,1648.408456,1134.087348,729.399389,404.687959,7847.199959,3165.222888,69.226118,987.865271,26.373124,974.895555,1179.529821,1444.087183,4606.232050,21.573892,27754.171585,12451.847931,11516.464166,1942.506619,2212.913746,73.085594,666.794729,372.006395,69.091752,123.058915,735.928397,330.471274,58.315733,110.083316,140.449118,3291.242626,1390.515952,913.490514,21.893250,412.421748,281.028347,131.393401,2177.161862,676.777184,127.794693,105.214681,1267.375304,1817.001560,1390.706105,83.757210,342.538245,1035.513213,667.938143,123.133925,244.441146,5377.127980,877.434475,35.249716,734.446778,2114.267115,259.943068,1854.324047,1615.729896,1816.644109,1196.157669,530.055144,90.431296,1770.918773,1417.940812,74.355416,278.622544,895.534408,4437.186994,1387.489851,755.125869,728.104925,17.474220,1548.992128,8474.901176,814.935448,468.138775,180.548120,287.590655,3192.599005,95.647992,1326.756290,1132.699423,878.485483,565.638760,88370.25,31
1025,31100,2022,540.5,2.0,1.0,2.332202,2.227814,49.155729,6.014349,3.493476,1.811269,1.682208,2.913202,0.580274,2.564022,0.349180,1.800049,2.380312,1.804270,59075.955870,40613.729112,35447.093311,31088.677550

In [76]:
# see 31 in table
table_concentradohogar_medians.query("cve_ent == '31'")

,cve_ent,year,est_dis,clase_hog,sexo_jefe,tam_loc,est_socio,edad_jefe,educa_jefe,tot_integ,hombres,mujeres,mayores,menores,p12_64,p65mas,ocupados,percep_ing,perc_ocupa,ing_cor,ingtrab,trabajo,sueldos,horas_extr,comisiones,aguinaldo,indemtrab,otra_rem,remu_espec,negocio,noagrop,industria,comercio,servicios,agrope,agricolas,pecuarios,reproducc,pesca,otros_trab,rentas,utilidad,arrenda,transfer,jubilacion,becas,donativos,remesas,bene_gob,transf_hog,trans_inst,estim_alqu,otros_ing,gasto_mon,alimentos,ali_dentro,cereales,carnes,pescado,leche,huevo,aceites,tuberculo,verduras,frutas,azucar,cafe,especias,otros_alim,bebidas,ali_fuera,tabaco,vesti_calz,vestido,calzado,vivienda,alquiler,pred_cons,agua,energia,limpieza,cuidados,utensilios,enseres,salud,atenc_ambu,hospital,medicinas,transporte,publico,foraneo,adqui_vehi,mantenim,refaccion,combus,comunica,educa_espa,educacion,esparci,paq_turist,personales,cuida_pers,acces_pers,otros_gas,transf_gas,percep_tot,retiro_inv,prestamos,otras_perc,ero_nm_viv,ero_nm_hog,erogac_tot,cuota_viv,mater_serv,material,servicio,deposito,prest_terc,pago_tarje,deudas,balance,otras_erog,total_hogares
90,31,2018,523.5,2.0,1.0,2.456703,2.136037,47.884917,5.393414,3.787951,1.928931,1.859019,3.043420,0.744530,2.769623,0.273798,2.135098,2.615438,2.119616,48829.923423,35268.822907,30158.612997,25160.049908,261.022478,1355.279482,1250.521360,95.521160,446.907764,1589.310846,3581.597498,3530.590390,885.469929,657.283153,1987.837307,51.007108,8.034997,3.786773,39.185338,0.0,1528.612412,3080.148049,2635.184312,444.963737,6337.237566,3092.755226,64.606151,969.174025,2.304668,405.076266,1361.925334,441.395896,4122.470070,21.244831,30109.082739,11960.566853,10201.588787,1727.076917,2238.691100,75.307536,552.903825,313.925006,76.572678,99.372963,757.031369,224.305888,62.460342,76.621581,132.800940,2424.238730,1440.279913,1721.087436,37.890630,988.841360,621.111680,367.729681,1814.376960,429.100939,54.296103,90.660146,1240.319772,1626.419767,1239.560925,81.348146,305.510696,881.197583,599.048439,173.890520,108.258624,6685.024772,2292.568324,104.673306,886.218412,1880.483958,163.920720,1716.563238,1521.080772,3374.304039,2388.266291,839.749936,146.287811,1805.657499,1510.164102,92.393330,203.100067,972.693908,3438.846770,945.507827,504.581552,643.013262,23.859404,1321.884725,7952.846139,956.530281,197.312960,130.386248,66.926712,3730.836279,378.061268,924.414087,623.302979,187.290210,955.098076,80021.50
91,31,2020,537.5,2.0,1.0,2.238077,2.122419,46.601857,5.963540,3.622582,1.704812,1.917770,2.841130,0.781453,2.586005,0.255125,1.875707,2.444548,1.814711,42069.208478,28460.115229,23616.203553,19266.157268,296.372213,1052.102110,1088.724343,214.578797,520.460090,1177.808734,3195.503220,3126.251192,503.937563,920.649766,1701.663863,69.252028,9.188451,29.590379,30.473198,0.0,1648.408456,1134.087348,729.399389,404.687959,7847.199959,3165.222888,69.226118,987.865271,26.373124,974.895555,1179.529821,1444.087183,4606.232050,21.573892,27754.171585,12451.847931,11516.464166,1942.506619,2212.913746,73.085594,666.794729,372.006395,69.091752,123.058915,735.928397,330.471274,58.315733,110.083316,140.449118,3291.242626,1390.515952,913.490514,21.893250,412.421748,281.028347,131.393401,2177.161862,676.777184,127.794693,105.214681,1267.375304,1817.001560,1390.706105,83.757210,342.538245,1035.513213,667.938143,123.133925,244.441146,5377.127980,877.434475,35.249716,734.446778,2114.267115,259.943068,1854.324047,1615.729896,1816.644109,1196.157669,530.055144,90.431296,1770.918773,1417.940812,74.355416,278.622544,895.534408,4437.186994,1387.489851,755.125869,728.104925,17.474220,1548.992128,8474.901176,814.935448,468.138775,180.548120,287.590655,3192.599005,95.647992,1326.756290,1132.699423,878.485483,565.638760,88370.25
92,31,2022,540.5,2.0,1.0,2.332202,2.227814,49.155729,6.014349,3.493476,1.811269,1.682208,2.913202,0.580274,2.564022,0.349180,1.800049,2.380312,1.804270,59075.955870,40613.729112,35447.093311,31088.677550,357.013911,629.566962,1316.879

### Vivendas

In [77]:
# to new_index add nan_ubica_geo ubica_geo's
index_cols = ["ubica_geo", "year"]
new_index = pd.MultiIndex.from_product(
    [df_properties["ubica_geo"].unique(), df_properties["year_enigh"].unique()],
    names=index_cols
)

# reindex
df_viviendas_all = (
    df_viviendas_all
    .set_index(index_cols)
    .reindex(new_index)
    .reset_index()
    )

In [78]:
# subset columns that are not duplicated
df_viviendas_all = (
    df_viviendas_all
    .loc[:, ~df_viviendas_all.columns.duplicated()]
)

In [79]:
# see 31100
df_viviendas_all.query("ubica_geo == '31100'")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
984,31100,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
985,31100,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
986,31100,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [80]:
# get cve_ent from ubica_geo
df_viviendas_all["cve_ent"] = df_viviendas_all["ubica_geo"].str[:2]

In [119]:
mask.sum()

2

In [125]:
# curate data error of '&' in tipo_finan
mask = df_viviendas_all['tipo_finan'].str.contains('&').astype(float).fillna(0).astype(bool)
# set to real nan
df_viviendas_all.loc[mask, 'tipo_finan'] = np.nan

In [126]:
# convert to float
cols_to_change_type = ["combustible", "tipo_finan", "tipo_viv", "mat_pisos"]
df_viviendas_all[cols_to_change_type] = df_viviendas_all[cols_to_change_type].astype(float)

In [127]:
# get table of median values by cve_ent
table_vivienda_medians = (
    df_viviendas_all
    .drop(columns=["ubica_geo"])
    .groupby(["cve_ent", "year"], as_index=False)
    .mean()
)
table_vivienda_medians

,cve_ent,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
0,01,2018,8.0,10.0,3.000000,1.0,3.0,3.0,4.00,2.000000,4.000000,1.0,1.0,1.0,0.920996,0.315568,0.805590,0.912256,0.0,4.305369,19.029272,1.130087,0.425857,0.965987,0.004484,2.139502,0.037438,275.551698,0.810831,1.663549,1.078883,0.039975,0.967105,2.414177,2.165868,1.155225,0.021514,0.004740,2.522576,0.098115,6.194716,0.348511,2.431610,1.066894,0.995430,0.088918,4.184356,0.425365,0.003381,0.254550,365.019505,0.297801,0.092614,0.014900,1.024408,1645.410893,71810.500000
1,01,2020,8.0,10.0,3.000000,1.0,3.0,3.0,4.00,1.500000,4.000000,1.0,1.0,1.0,0.948013,0.301067,0.829743,0.904102,0.0,3.918506,17.347477,1.154849,0.446635,0.987648,0.007158,1.912456,0.042310,305.124306,0.802218,1.105541,1.084915,0.029291,0.974117,2.364563,2.006050,1.136489,0.014086,0.004851,2.449221,0.118582,6.506671,0.353960,2.296961,1.052942,0.992563,0.118789,4.287628,0.406414,0.005875,0.191794,389.388663,0.320350,0.115512,0.008485,1.015404,1691.097447,84977.750000
2,01,2022,8.0,10.0,3.000000,1.0,3.0,3.0,3.75,1.500000,3.000000,1.0,1.0,1.0,0.953315,0.363934,0.833432,0.888370,0.0,3.862661,17.033339,1.187011,0.527365,0.964534,0.017653,1.864706,0.024552,350.099027,0.856483,0.747706,1.093178,0.036064,0.962341,2.337955,1.997955,1.259933,0.010746,0.004450,2.679804,0.120712,7.343714,0.335790,2.291607,1.070470,0.990484,0.112913,4.165880,0.404203,0.010323,0.164826,589.775208,0.363320,0.116361,0.022477,1.018898,2183.065867,85916.500000
3,02,2018,8.0,8.5,2.250000,1.0,3.0,1.0,4.00,1.500000,5.000000,1.0,1.0,1.0,0.852444,0.313906,0.101366,0.603249,0.0,3.370514,14.778483,0.976271,0.013631,0.974302,0.271139,1.682316,0.068284,495.536871,0.822640,1.190902,1.249026,0.059280,0.934396,2.450202,1.688198,0.994116,0.034741,0.002962,1.786824,0.050373,5.619687,0.042568,2.115245,1.394415,0.995097,0.127318,3.651680,0.542618,0.030859,0.331060,355.509805,0.063110,0.130529,0.006312,1.003140,2137.880791,266591.000000
4,02,2020,8.0,8.5,2.500000,1.0,3.0,1.0,4.00,2.000000,5.000000,1.0,1.0,1.0,0.913127,0.279764,0.108240,0.577239,0.0,3.309996,18.097982,1.099012,0.025186,0.979650,0.272621,1.613549,0.072284,519.522103,0.860794,0.819700,1.161200,0.088035,0.964287,2.485101,1.696447,1.115911,0.031589,0.001947,1.784814,0.062003,6.346981,0.043219,2.161368,1.252475,0.994902,0.170105,3.842881,0.588295,0.044019,0.279955,386.555622,0.059437,0.128789,0.021931,1.005444,2910.766735,272823.250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,31,2020,8.0,10.0,3.000000,1.0,3.0,1.0,4.00,1.250000,3.000000,1.0,1.0,1.0,0.882248,0.319899,0.834266,0.606312,0.0,3.697294,15.621745,0.997795,0.003348,0.936638,0.252943,1.737912,0.095735,236.960118,0.792539,0.351484,1.211693,0.042477,0.948645,2.122419,1.959382,0.987243,0.005081,0.011412,2.238077,0.023373,7.078678,0.090151,1.859020,2.074360,0.979938,0.072890,3.527309,0.103532,0.002434,0.144663,368.438554,0.255339,0.097741,0.074711,1.025997,1566.595569,88370.250000
92,31,2022,8.0,10.0,2.750000,1.0,3.0,1.0,4.00,1.250000,2.000000,1.0,1.0,1.0,0.888618,0.283053,0.861815,0.624339,0.0,3.546514,20.736073,1.021109,0.004833,0.939103,0.340484,1.842908,0.105069,160.820769,0.776416,0.157252,1.120146,0.021508,0.954367,2.227814,1.703606,1.004846,0.004446,0.001194,2.332202,0.039448,7.641070,0.096652,1.845370,1.972431,0.979488,0.101510,3.484885,0.098103,0.

In [129]:
# append to df_viviendas_all for nan values
mask = (
    df_viviendas_all['mat_pared'].isnull()
)
columns_to_impute = df_viviendas_all.drop(columns=["ubica_geo", "year", "cve_ent"]).columns.copy()

# append to df_viviendas_all for nan values
df_viviendas_all.loc[mask, columns_to_impute] = (
    df_viviendas_all
    .loc[mask, :]
    .merge(
        table_vivienda_medians,
        on=["cve_ent", "year"],
        how="left"
        )
    .filter(like="_y")
    .values
)

# sort by ubica_geo and year
df_viviendas_all = (
    df_viviendas_all
    .sort_values(["ubica_geo", "year"], ignore_index=True)
    )

# see how many nan values
df_viviendas_all.isnull().sum()[df_viviendas_all.isnull().sum() > 0]

tipo_finan    2
dtype: int64

In [130]:
# see 31100
df_viviendas_all.query("ubica_geo == '31100'")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,cve_ent
1023,31100,2018,8.0,10.0,2.75,1.0,2.5,1.0,3.75,1.25,3.0,1.0,1.0,1.0,0.911595,0.267145,0.868114,0.485174,0.0,3.949940,19.816853,1.027697,0.002856,0.833350,0.230294,2.014664,0.086792,145.350302,0.733703,0.660290,1.103274,0.018584,0.981393,2.136037,1.935276,0.982264,0.002450,0.023236,2.456703,0.040776,6.723034,0.094318,1.946893,2.028594,0.968858,0.04332,3.485306,0.143839,0.000466,0.108094,345.033435,0.165134,0.138853,0.100664,1.048106,1405.562140,80021.50,31
1024,31100,2020,8.0,10.0,3.00,1.0,3.0,1.0,4.00,1.25,3.0,1.0,1.0,1.0,0.882248,0.319899,0.834266,0.606312,0.0,3.697294,15.621745,0.997795,0.003348,0.936638,0.252943,1.737912,0.095735,236.960118,0.792539,0.351484,1.211693,0.042477,0.948645,2.122419,1.959382,0.987243,0.005081,0.011412,2.238077,0.023373,7.078678,0.090151,1.859020,2.074360,0.979938,0.07289,3.527309,0.103532,0.002434,0.144663,368.438554,0.255339,0.097741,0.074711,1.025997,1566.595569,88370.25,31
1025,31100,2022,8.0,10.0,2.75,1.0,3.0,1.0,4.00,1.25,2.0,1.0,1.0,1.0,0.888618,0.283053,0.861815,0.624339,0.0,3.546514,20.736073,1.021109,0.004833,0.939103,0.340484,1.842908,0.105069,160.820769,0.776416,0.157252,1.120146,0.021508,0.954367,2.227814,1.703606,1.004846,0.004446,0.001194,2.332202,0.039448,7.641070,0.096652,1.845370,1.972431,0.979488,0.10151,3.484885,0.098103,0.004757,0.134482,529.623155,0.129257,0.165997,0.069498,1.017018,2088.318963,87237.25,31


In [131]:
# see 31 in table
table_vivienda_medians.query("cve_ent == '31'")

,cve_ent,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas
90,31,2018,8.0,10.0,2.75,1.0,2.5,1.0,3.75,1.25,3.0,1.0,1.0,1.0,0.911595,0.267145,0.868114,0.485174,0.0,3.949940,19.816853,1.027697,0.002856,0.833350,0.230294,2.014664,0.086792,145.350302,0.733703,0.660290,1.103274,0.018584,0.981393,2.136037,1.935276,0.982264,0.002450,0.023236,2.456703,0.040776,6.723034,0.094318,1.946893,2.028594,0.968858,0.04332,3.485306,0.143839,0.000466,0.108094,345.033435,0.165134,0.138853,0.100664,1.048106,1405.562140,80021.50
91,31,2020,8.0,10.0,3.00,1.0,3.0,1.0,4.00,1.25,3.0,1.0,1.0,1.0,0.882248,0.319899,0.834266,0.606312,0.0,3.697294,15.621745,0.997795,0.003348,0.936638,0.252943,1.737912,0.095735,236.960118,0.792539,0.351484,1.211693,0.042477,0.948645,2.122419,1.959382,0.987243,0.005081,0.011412,2.238077,0.023373,7.078678,0.090151,1.859020,2.074360,0.979938,0.07289,3.527309,0.103532,0.002434,0.144663,368.438554,0.255339,0.097741,0.074711,1.025997,1566.595569,88370.25
92,31,2022,8.0,10.0,2.75,1.0,3.0,1.0,4.00,1.25,2.0,1.0,1.0,1.0,0.888618,0.283053,0.861815,0.624339,0.0,3.546514,20.736073,1.021109,0.004833,0.939103,0.340484,1.842908,0.105069,160.820769,0.776416,0.157252,1.120146,0.021508,0.954367,2.227814,1.703606,1.004846,0.004446,0.001194,2.332202,0.039448,7.641070,0.096652,1.845370,1.972431,0.979488,0.10151,3.484885,0.098103,0.004757,0.134482,529.623155,0.129257,0.165997,0.069498,1.017018,2088.318963,87237.25


## S6: Re-Join with df_properties

### Load & Transform

In [132]:
# read parquet
df_properties = pd.read_parquet("../../data/interim/cleaned_data_s4.parquet")

# get original columns
original_columns = df_properties.columns

# see num of rows
df_properties.shape

(1909959, 51)

In [133]:
# see columns cve ent & cve mun
df_properties[["state_id", "municipality_id"]]

,state_id,municipality_id
0,31,050
1,31,041
2,31,050
3,31,041
4,31,050
...,...,...
1909954,01,001
1909955,01,001
1909956,14,120
1909957,13,013


In [134]:
# create 'ubica_geo' column
df_properties["ubica_geo"] = (
    df_properties["state_id"].astype(str).str.zfill(2)
    + df_properties["municipality_id"].astype(str).str.zfill(3)
    )

# see some values
df_properties["ubica_geo"]

0          31050
1          31041
2          31050
3          31041
4          31050
           ...  
1909954    01001
1909955    01001
1909956    14120
1909957    13013
1909958    15081
Name: ubica_geo, Length: 1909959, dtype: object

In [135]:
# get year of fecha avaluo and floor every 2 years
df_properties["year_enigh"] = (
    df_properties["valuation_date"].dt.year
    // 2 * 2
    )

# describe
df_properties["year_enigh"].describe()

count    1.909959e+06
mean     2.020327e+03
std      1.499379e+00
min      2.018000e+03
25%      2.020000e+03
50%      2.020000e+03
75%      2.022000e+03
max      2.022000e+03
Name: year_enigh, dtype: float64

### Join with Concentrado Hogar

In [136]:
# join with df_concentradohogar_all
df_properties = (
    df_properties
    .merge(
        df_concentradohogar_all,
        left_on=["ubica_geo", "year_enigh"],
        right_on=["ubica_geo", "year"],
        how="left"
        )
    )

# see columns
df_properties.shape

(1909959, 176)

In [137]:
# see size
df_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1909959 entries, 0 to 1909958
Columns: 176 entries, half_bathrooms to cve_ent
dtypes: datetime64[ns](1), float32(3), float64(137), int16(1), int64(2), object(4), string(12), uint16(1), uint32(1), uint8(14)
memory usage: 2.3+ GB


In [138]:
# see if there are columns with "_y"
df_properties.filter(like="_x").columns

Index([], dtype='object')

In [139]:
# see how many nan values 
df_properties.isnull().sum()[df_properties.isnull().sum() > 0]

neighborhood                           118
constructor                           2518
potable_water_service_id            633139
lighting_service_id                 633119
sidewalk_id                         634669
project_quality_id                  647455
elevator_service_id                  91867
curb_id                             633119
water_collection_service_id         633119
electrical_supply_service_id        633119
telephone_service_supply_id         633119
public_transportation_service_id    633119
rentable_units                      265108
rentable_units_subject_property     647450
current_use                         641538
access_routes                       633173
dtype: int64

### Join with Vivienda

In [140]:
# join with df_viviendas_all
df_properties = (
    df_properties
    .drop(columns=["year", "tam_loc", "est_socio"])
    .merge(
        df_viviendas_all,
        left_on=["ubica_geo", "year_enigh"],
        right_on=["ubica_geo", "year"],
        how="left"
        )
    )

# see columns
df_properties.shape

(1909959, 230)

In [141]:
# see size
df_properties.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1909959 entries, 0 to 1909958
Columns: 230 entries, half_bathrooms to cve_ent_y
dtypes: datetime64[ns](1), float32(3), float64(190), int16(1), int64(2), object(5), string(12), uint16(1), uint32(1), uint8(14)
memory usage: 3.0+ GB


In [142]:
# see if there are columns with "_y"
df_properties.filter(like="_x").columns

Index(['cve_ent_x'], dtype='object')

In [143]:
# drop cve_ent_x and cve_ent_y
df_properties = (
    df_properties
    .drop(columns=["cve_ent_x", "cve_ent_y"])
    )

In [144]:
# see how many nan values 
df_properties.isnull().sum()[df_properties.isnull().sum() > 0]

neighborhood                           118
constructor                           2518
potable_water_service_id            633139
lighting_service_id                 633119
sidewalk_id                         634669
project_quality_id                  647455
elevator_service_id                  91867
curb_id                             633119
water_collection_service_id         633119
electrical_supply_service_id        633119
telephone_service_supply_id         633119
public_transportation_service_id    633119
rentable_units                      265108
rentable_units_subject_property     647450
current_use                         641538
access_routes                       633173
tipo_finan                               9
dtype: int64

---
# Save

In [145]:
# save
dir_save = "../../data/interim/"
df_viviendas_all.to_csv(dir_save + "viviendas_enigh.csv")
df_concentradohogar_all.to_csv(dir_save + "concentrado_hogar_enigh.csv")

In [146]:
# save parquet
df_properties.to_parquet("../../data/interim/cleaned_data_s5_adamuz.parquet")

---
# Sandbox

In [44]:
df_viviendas_all.columns.duplicated()

array([False, False, False, False, False, False, False, False, False,
       False,  True, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False])

In [41]:
df_viviendas_all.dtypes

ubica_geo           object
year                 int64
mat_pared          float64
mat_techos         float64
mat_pisos           object
disp_agua          float64
combustible         object
eli_basura         float64
tenencia           float64
tipo_adqui         float64
tipo_adqui         float64
tipo_finan          object
escrituras         float64
disp_elect         float64
tipo_viv            object
regadera           float64
pago_mesp          float64
tinaco_azo         float64
lavadero           float64
procaptar          float64
tot_resid          float64
antiguedad         float64
bano_comp          float64
calent_sol         float64
cocina             float64
aire_acond         float64
tot_hom            float64
cocina_dor         float64
renta              float64
fregadero          float64
focos_inca         float64
sanit_agua         float64
uso_compar         float64
medidor_luz        float64
est_socio          float64
tot_muj            float64
dotac_agua         float64
b

In [40]:
df_viviendas_all['ubica_geo'].dtype

dtype('O')

In [109]:
# df_viviendas_all.query("tipo_finan.str.contains('&')")
df_viviendas_all[df_viviendas_all['tipo_finan'].str.contains('&').astype(float).fillna(0).astype(bool)]

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,cve_ent
923,23003,2018,8.0,10.0,2.0,7.0,3,1.0,3.0,2.0,&,1.0,1.0,1,0.2,0.6,0.2,0.4,0.0,4.0,3.8,0.4,0.0,1.0,0.6,2.6,0.4,200.0,0.4,0.0,1.8,0.2,0.8,1.0,1.4,0.0,0.0,0.0,3.0,0.0,5.4,0.0,1.2,2.0,1.0,0.6,1.8,0.0,0.0,0.8,1360.0,1.0,0.0,0.0,1.0,1040.0,2575.0,23
992,20083,2018,8.0,10.0,3.0,6.0,3.0,1.0,4.0,1.0,&,1.0,1.0,1,1.0,0.2,0.8,0.8,0.0,4.8,21.2,1.2,0.2,1.0,0.0,2.6,0.0,200.0,1.0,0.0,1.8,0.0,1.0,2.0,2.2,1.4,0.0,0.0,3.0,0.2,10.8,0.8,2.6,2.6,1.0,0.0,4.2,0.8,0.0,0.2,660.0,0.8,0.2,0.0,1.0,5000.0,3905.0,20


In [108]:
df_viviendas_all['tipo_finan'].str.contains('&').astype(float).fillna(0).astype(bool)

0       False
1       False
2       False
3       False
4       False
        ...  
1033    False
1034    False
1035    False
1036    False
1037    False
Name: tipo_finan, Length: 1038, dtype: bool

In [115]:
df_viviendas_all['combustible'].astype(float)

0       3.0
1       3.0
2       3.0
3       3.0
4       3.0
       ... 
1033    3.0
1034    3.0
1035    NaN
1036    NaN
1037    NaN
Name: combustible, Length: 1038, dtype: float64

In [84]:
df_viviendas_all.query("ubica_geo.eq('01004')")

,ubica_geo,year,mat_pared,mat_techos,mat_pisos,disp_agua,combustible,eli_basura,tenencia,tipo_adqui,tipo_finan,escrituras,disp_elect,tipo_viv,regadera,pago_mesp,tinaco_azo,lavadero,procaptar,tot_resid,antiguedad,bano_comp,calent_sol,cocina,aire_acond,tot_hom,cocina_dor,renta,fregadero,focos_inca,sanit_agua,uso_compar,medidor_luz,est_socio,tot_muj,dotac_agua,bano_regad,estufa_chi,tam_loc,tanque_gas,focos_ahor,cisterna,cuart_dorm,drenaje,excusado,pileta,num_cuarto,calent_gas,calefacc,bano_excus,pago_viv,bomba_agua,viv_usada,biodigest,tot_hog,estim_pago,total_viviendas,cve_ent
